# Use pearson correlation coefficient (for linear regression) as a loss function

A wild try: when we try to predict a value where random noise is involved, it might be a good idea to use the regression coefficient for a linear regression between our prediction and the real value as a loss function and metric. In the ideal case, the prediction would be equal to the real value, we'd have a regression coefficient 1.

Of course, the regression coefficient leaves too much freedom for the prediction: the prediction can be any linear function of the real value. So we have to introduce an additional loss, such that these are close. There are several ways: we could add the squared difference of the means and variations of both distributions multiplied with some low factor, to make sure they are equal. Or we could add the "conventional" mean squared difference multiplied with a low factor. You name it.


I'm not sure how much sense that makes, but let's try. It might work well if we try to predict something that is not far from linear but with lots of noise.

In [0]:
#@title Imports and utilities {display-mode: "form"}


import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import numpy as np
import functools
import itertools
import time
import os
import datetime
import re
import altair as alt

print([tf.__version__, pd.__version__, sns.__version__ ,plt.matplotlib.__version__,np.__version__])

from IPython.core.pylabtools import figsize
figsize(12.5, 3)

# Display training progress by printing a single dot for each completed epoch
class PrintDot(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, min_delta=0.001, verbose=1, restore_best_weights=True)
terminate_nan = keras.callbacks.TerminateOnNaN()

def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Val Loss')
    plt.semilogy(hist['epoch'], hist['loss'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_loss'], label='Val Error')
    # plt.ylim([0,5])
    plt.legend()
    
def meanAndVariance(y_true, y_pred):
  y_pred = tf.convert_to_tensor(y_pred)
  y_true = math_ops.cast(y_true, y_pred.dtype)
  means = y_pred[..., 0::2]
  variances = y_pred[..., 1::2]
  res = K.square(means - y_true) + K.square(variances - K.square(means - y_true))
  return K.mean(res, axis=-1)

def nans(ds):
  return ds.loc[ds.isna().any(axis=1)]

def plothisto(df, col=None):
  """Plots an interactive histogram of a (default first) column of a dataset"""
  col = col if (col) else df.columns[0]
  return alt.Chart(df.sample(5000).reset_index()).mark_bar().encode(
    x=alt.X(col, bin= alt.BinParams(maxbins=200) ),
    y='count()',
  ).interactive(bind_y=False)

%config IPCompleter.greedy=True

In [0]:
tf.enable_eager_execution()
#%xmode Verbose
#%xmode Context

def t(a):
  """For testing: generate a float64 tensor from anything."""
  return tf.constant(a, dtype=tf.float64)

In [0]:
def tmean(x, axis=-1):
  """Arithmetic mean of a tensor over some axis, default last."""
  x = tf.convert_to_tensor(x)
  sum = tf.reduce_sum(x, axis=axis)
  n = tf.cast(tf.shape(x)[axis], x.dtype)
  return sum / n

tmean(t([[1.0],[2.0],[3.0]]), axis=-2)

Pearson correlation coefficlent : $r_{xy} = \frac{\sum\left((x-\overline{x})(y-\overline{y})\right)}{\sqrt{\sum(x-\overline{x})^2\sum(y-\overline{y})^2}} $

In [0]:
def correlationMetric(x, y, axis=-2):
  """Metric returning the Pearson correlation coefficient of two tensors over some axis, default -2."""
  x = tf.convert_to_tensor(x)
  y = math_ops.cast(y, x.dtype)
  n = tf.cast(tf.shape(x)[axis], x.dtype)
  xsum = tf.reduce_sum(x, axis=axis)
  ysum = tf.reduce_sum(y, axis=axis)
  xmean = xsum / n
  ymean = ysum / n
  xvar = tf.reduce_sum( tf.squared_difference(x, xmean), axis=axis)
  yvar = tf.reduce_sum( tf.squared_difference(y, ymean), axis=axis)
  cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
  corr = cov / tf.sqrt(xvar * yvar)
  return tf.constant(1.0, dtype=x.dtype) - corr

correlationMetric(tf.constant([[0.0, 1.0, 2.0]]), tf.constant([[1.0, 3.0, 2.0]]), axis=-1)
correlationMetric(tf.constant([[0.0, 2.0, 1.0]]), tf.constant([[1.0, 3.0, 2.0]]), axis=-1)
correlationMetric(tf.constant([[0.0], [2.0], [1.0]]), tf.constant([[1.0], [3.0], [2.0]]), axis=-2)

In [0]:
def correlationLoss(x,y, axis=-2):
  """Loss function that maximizes the pearson correlation coefficient between the predicted values and the labels,
  while trying to have the same mean and variance"""
  x = tf.convert_to_tensor(x)
  y = math_ops.cast(y, x.dtype)
  n = tf.cast(tf.shape(x)[axis], x.dtype)
  xsum = tf.reduce_sum(x, axis=axis)
  ysum = tf.reduce_sum(y, axis=axis)
  xmean = xsum / n
  ymean = ysum / n
  xsqsum = tf.reduce_sum( tf.squared_difference(x, xmean), axis=axis)
  ysqsum = tf.reduce_sum( tf.squared_difference(y, ymean), axis=axis)
  cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
  corr = cov / tf.sqrt(xsqsum * ysqsum)
  # absdif = tmean(tf.abs(x - y), axis=axis) / tf.sqrt(yvar)
  sqdif = tf.reduce_sum(tf.squared_difference(x, y), axis=axis) / n / tf.sqrt(ysqsum / n)
  # meandif = tf.abs(xmean - ymean) / tf.abs(ymean)
  # vardif = tf.abs(xvar - yvar) / yvar
  # return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr + (meandif * 0.01) + (vardif * 0.01)) , dtype=tf.float32 )
  return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr + (0.01 * sqdif)) , dtype=tf.float32 )

#t1,t2 = tf.constant([[0.0], [3.0], [2.0]]), tf.constant([[0.0], [1.0], [3.0]])
#print(correlationLoss(t1,t2))
#print(correlationMetric(t1,t2))
#del t1,t2

In [0]:
samples = 10240
epochs = 100
steps = 1024
batchsize = 128
validationsamples = samples // 10

As testdata we use a simple example with gaussian random inputs x_d and r_d ( \_d is a postfix that marks the column as data to learn from), some nonlinear function to approximate and add a little noise dependent on the inputs. The postfix \_g designates the column as "goal" to approximate.

In [0]:
data = pd.DataFrame( np.random.randn(samples,2), columns=['x_d', 'r_d'] ).sample(frac=1.0)
data['y_g'] = data.x_d * data.x_d + 1
data.y_g = data.y_g + np.random.randn(samples) * ( np.cos(3*data.x_d) ) * 0.5
data['z_g'] = data.x_d * data.r_d + 3
data.z_g = data.z_g + np.random.randn(samples) * data.x_d * data.r_d * 0.1

train_data = data.filter(regex='_d').to_numpy()
train_labels = data.filter(regex='_g').to_numpy()
print(train_data.shape, train_labels.shape)
data.sample(3)

In [0]:
traindataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
traindataset = traindataset.shuffle(samples)

validationdataset = traindataset.take(validationsamples)
traindataset = traindataset.skip(validationsamples)

traindataset = traindataset.repeat().batch(batchsize, drop_remainder=True)
validationdataset = validationdataset.repeat().batch(batchsize, drop_remainder=True)
# traindataset.make_one_shot_iterator().get_next()
# validationdataset.make_one_shot_iterator().get_next()

In [0]:
# loss = tf.losses.mean_squared_error
loss = correlationLoss

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, activation=tf.nn.softsign))
model.add(tf.keras.layers.Dense(32, activation=tf.nn.softsign))
model.add(tf.keras.layers.Dense(2))
# just a linear transformation that makes it easy to scale / shift the prediction right
# model.add(tf.keras.layers.Dense(2))
model.compile(loss=loss , optimizer='adam', metrics=[loss, correlationMetric, "mse", "mae"])

model.fit(traindataset, steps_per_epoch=1, epochs=1) # this initializes the input_shape
model.summary()

In [0]:
%time history = model.fit(traindataset, epochs=epochs, steps_per_epoch=steps, validation_data=validationdataset, validation_steps=validationsamples // batchsize, callbacks=[terminate_nan, early_stop])
#%time history = model.fit(tdataset, epochs=epochs, steps_per_epoch=steps, validation_split=0.1, callbacks=[terminate_nan])

In [0]:
print(history.history['loss'][-1], np.sqrt(history.history['loss'][-1]))
print(history.history['val_loss'][-1], np.sqrt(history.history['val_loss'][-1]))
plot_history(history)
# print(datetime.datetime.now())
pd.DataFrame(history.history).plot(logy=True, figsize=(12.5,12.5))
model.summary()
history.params

In [0]:
p = pd.DataFrame(model.predict(data.filter(regex='_d').to_numpy()))
p.columns = data.filter(regex='_g').columns.str.replace('_g','_p')
p.index = data.index
data = data.join(p)
data['y_o'] = data.y_p - data.y_g  # "offset"
data['z_o'] = data.z_p - data.z_g 
data.sample(3)

# Conclusion

It seems to work, indeed, though it converges in our example more slowly than, for example, mean squared errors. The next diagram shows the prediction, which looks very much like the real value below. The third diagram shows the difference.

I'm not sure how much it's worth, but this might be another tool in our toolbox if there is randomness involved.

In [0]:
# let's see how we are doing on the correlation
data.corr()

In [0]:
alt.Chart(data.sample(1000).reset_index()).mark_point(size=0.1).encode(
  x='x_d',
  y='y_p'
).interactive()

In [0]:
alt.Chart(data.sample(1000).reset_index()).mark_point(size=0.1).encode(
  x='x_d',
  y='y_g'
).interactive()

In [0]:
alt.Chart(data.sample(1000).reset_index()).mark_point(size=0.1).encode(
  x='x_d',
  y='y_o'
).interactive()

In [0]:
# Interesting is also how the difference depends on p - which is what we correlate

alt.Chart(data.sample(1000).reset_index()).mark_point(size=0.1).encode(
  x='y_p',
  y='y_o'
).interactive()

In [0]:
# that's the actual data on which we maximize the correlation

alt.Chart(data.sample(1000).reset_index()).mark_point(size=0.1).encode(
  x='y_p',
  y='y_g'
).interactive()